### 忘记这页是干啥的了。。

In [27]:
# coding: utf-8

from logger import getLogger
import pymysql
from config import MYSQL_DEVELOPMENT

logger = getLogger(__name__)
conn = pymysql.connect(
    **MYSQL_DEVELOPMENT,
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=True
)
cursor = conn.cursor()


def get(es_index, es_id):
    from es import es
    data = es.get(es_index, 'news', es_id, ignore=400, _source=['title', 'content'])

    _source = data['_source']
    return {
        'es_id': es_id,
        'es_index': es_index,
        'content': _source['content'],
        'title': _source['title']
    }


# 拆分为句子
def sentencesMaker(html):
    sentences = []
    if not html or not html.strip():
        return sentences
    try:
        from html.parser import unescape
        html = unescape(html)

        import justext
        paragraphs = justext.justext(html, [])

        cache_sentences = ''

        for p in paragraphs:
            sent = p.text.strip().replace('\xa0', '').replace('\u3000', '')
            sent = sent.encode('gb2312', 'ignore').decode('gb2312').encode('gbk', 'ignore').decode('gbk')
            if not sent:
                continue

            # 可能是含有名字，需要进一步处理
            if len(cache_sentences) < 5:
                cache_sentences += ' ' + sent
            else:
                sentences.append(cache_sentences.strip())
                cache_sentences = sent

        if not not cache_sentences:
            sentences.append(cache_sentences.strip())
    except Exception as e:
        logger.error(e)

    return sentences


# dlist = ["36kr"]  # "36kr", , "JinRiTouTiao", "Wechat"

if __name__ == '__main__':
    i = 1
    with open("./TouTiao_testdata1_datalist.txt", "r") as output:
        for line in output:
            if line:
                try:
                    es_id = line.strip()
                    es_index = "toutiao_bl2"
                    data = get(es_index, es_id)
                    with open("./TouTiao_testdata1/%s.txt" % i, "w", encoding="utf-8") as output:
                        output.write(data['title'] + "\n")
                        output.write("，".join(sentencesMaker(data['content'])) + "\n")
                        i += 1
                except Exception as e:
                    print("./TouTiao_testdata1/%s.txt" % i)

GET http://123.206.13.101:59200/toutiao_bl2/news/6585014543854862852?_source=title%2Ccontent [status:404 request:0.041s]


./TouTiao_testdata1/7.txt


In [28]:
'''
    查看共取下来多少篇文章，有的可能数据库中就是空的
'''
import os
print(len(os.listdir("./TouTiao_testdata1")))

199


In [29]:
'''
    在取下来的每篇文章最后一行加上类别标识，1/0
'''
import os
for file in os.listdir("./TouTiao_testdata1"):
    if file.split(".")[0].isdigit():
        data = open("./TouTiao_testdata1/%s" % file, "a", encoding="utf-8")
        data.write("0")
        data.close()

In [30]:
'''
    查看有多少篇文章不是三行格式
'''
import os
number = []
for file in os.listdir("./TouTiao_testdata1"):
    if file.split(".")[0].isdigit():
        f = open("./TouTiao_testdata1/%s" % file, "r", encoding="utf-8")
        data = f.readlines()
        if len(data) != 3:
            number.append(file)
            print("./TouTiao_testdata1/%s" % file)
            print(len(data))
            print("*******************************************************")
        f.close()
print(number)
print(len(number))

[]
0


In [31]:
'''
    将非三行格式的数据删除
'''
import os

for file in os.listdir("./TouTiao_testdata1"):
    if file in number:
        os.remove("./TouTiao_testdata1/%s" % file)

In [32]:
'''
    训练集中新加入5227个负样本
'''
print(len(os.listdir("./TouTiao_testdata1")))

199


In [33]:
import os

inputs = open("./TouTiao_testdata1_format.txt", "w", encoding="utf-8")

for file in os.listdir("./TouTiao_testdata1"):
#     print(len(os.listdir("./Wechat_negative1")))
    if file.split(".")[0].isdigit():
        data = open("./TouTiao_testdata1/%s" % file, "r", encoding="utf-8")
        text = []
        i = 1
        label = ""
        for line in data:
            if line and i < 3:
                text.append(line.strip())
                i += 1
            elif line and i == 3:
                label = line
            elif line and i > 3:
                print("***************************")
                print("./TouTiao_testdata1/%s" % file)
                print("***************************")
#         print("。".join(text))

        inputs.write(str(label) + "\t" +"。".join(text) + "\n")